<a href="https://colab.research.google.com/github/greapis/jaeha-shim/blob/main/Fourth%20ML%20model(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 컨볼루션을 통한 컴퓨터 비전의 정확성 개선

# 이제 심층 신경망(DNN)을 사용하여 입력 레이어(입력 데이터 모양)와
# 히든 레이어의 세 가지 레이어로 구성된 패션 이미지 인식을 수행하는 방법을 알게 되었습니다.

# 히든 레이어의 크기, 학습 세대의 수와 같이 최종 정확성에 영향을 주는 여러 매개변수를 실험했습니다.

# 편의를 위해 전체 코드를 다시 보내드립니다. 이를 실행하고 끝에 출력된 테스트 정확성을 기록해 둡니다.

In [8]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images/255.0
test_images = test_images/255.0
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print('Test loss: {}, Test accuracy: {}'.format(test_loss, test_accuracy*100))

# 컨볼루션을 사용하면 이미지의 내용을 좁혀 구체적인 세부정보에 초점을 맞출 수 있습니다.

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5006 - accuracy: 0.8257
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3740 - accuracy: 0.8668
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3351 - accuracy: 0.8782
Epoch 4/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3120 - accuracy: 0.8855
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3383 - accuracy: 0.8799
Test loss: 0.3383041024208069, Test accuracy: 87.98999786376953


In [14]:
## 3. 코드 사용해 보기
# 다음 코드를 실행합니다. 이전의 신경망과 동일하지만 이번에는 컨볼루셔널 레이어가 먼저 추가됩니다.
# 시간이 좀 더 걸리지만 정확성에 미치는 영향을 살펴봅니다.


import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images/255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images/255.0
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print('Test loss: {}, Test accuracy: {}'.format(test_loss, test_accuracy*100))

# 학습 데이터에 대해 약 93%, 검증 데이터에서 약 91% 의 상승 효과를 보였습니다.

# 이제 더 많은 에포크 동안(예: 20개 정도) 결과를 실행해 보세요.
# 학습 결과는 매우 좋아 보일 수 있지만 과적합이라는 현상으로 인해 유효성 검사 결과가 실제로 낮아질 수 있습니다.

2.15.0
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_10 (Dense)            (None, 128)       

In [17]:
## 4. 데이터 수집
# 첫 번째 단계는 데이터를 수집하는 것입니다.

# 여기서 달라진 부분과 학습 데이터를 다시 구성하는 데 필요한 학습 데이터가 있음을 알 수 있습니다.
# 첫 번째 컨볼루션에서는 모든 것을 포함하는 단일 텐서가 예상되므로,
# 목록에 있는 20x28x1 항목 60,000개 대신 60,000x28x28x1인 테스트용 단일 4D 목록이 있습니다.
# 이렇게 하지 않으면 컨볼루션이 모양을 인식하지 못하므로 학습할 때 오류가 발생합니다.


import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_image, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images.reshape(60000, 28, 28, 1)
training_image = training_images/255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images/255.0

In [ ]:
## 5. 모델 정의
# 다음으로 모델을 정의합니다. 상단의 입력 레이어 대신 컨볼루셔널 레이어를 추가합니다.
# 사용할 수 있는 매개변수는 다음과 같습니다.

# 생성하려는 컨볼루션의 수입니다. 32와 같은 값은 좋은 출발점입니다.
# 컨볼루셔널 행렬의 크기(이 경우 3x3 그리드)
# 사용할 활성화 함수입니다. 이 경우에는 relu를 사용합니다.
# 첫 번째 레이어에서 입력 데이터의 모양입니다.